In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [61]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_HistogramAMI(y, tau = 1, meth = 'even', numBins = 10):
    """
    CO_HistogramAMI: The automutual information of the distribution using histograms.

    Parameters:
    y (array-like): The input time series
    tau (int, list or str): The time-lag(s) (default: 1)
    meth (str): The method of computing automutual information:
                'even': evenly-spaced bins through the range of the time series,
                'std1', 'std2': bins that extend only up to a multiple of the
                                standard deviation from the mean of the time series to exclude outliers,
                'quantiles': equiprobable bins chosen using quantiles.
    num_bins (int): The number of bins (default: 10)

    Returns:
    float or dict: The automutual information calculated in this way.
    """
    # Use first zero crossing of the ACF as the time lag
    if isinstance(tau, str) and tau in ['ac', 'tau']:
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Bins for the data
    # same for both -- assume same distribution (true for stationary processes, or small lags)
    if meth == 'even':
        b = np.linspace(np.min(y), np.max(y), numBins + 1)
        # Add increment buffer to ensure all points are included
        inc = 0.1
        b[0] -= inc
        b[-1] += inc
    elif meth == 'std1': # bins out to +/- 1 std
        b = np.linspace(-1, 1, numBins + 1)
        if np.min(y) < -1:
            b = np.concatenate(([np.min(y) - 0.1], b))
        if np.max(y) > 1:
            b = np.concatenate((b, [np.max(y) + 0.1]))
    elif meth == 'std2': # bins out to +/- 2 std
        b = np.linspace(-2, 2, numBins + 1)
        if np.min(y) < -2:
            b = np.concatenate(([np.min(y) - 0.1], b))
        if np.max(y) > 2:
            b = np.concatenate((b, [np.max(y) + 0.1]))
    elif meth == 'quantiles': # use quantiles with ~equal number in each bin
        b = np.quantile(y, np.linspace(0, 1, numBins + 1))
        b[0] -= 0.1
        b[-1] += 0.1
    else:
        raise ValueError(f"Unknown method '{meth}'")
    
    # Sometimes bins can be added (e.g., with std1 and std2), so need to redefine numBins
    numBins = len(b) - 1

    # Form the time-delay vectors y1 and y2
    if not isinstance(tau, (list, np.ndarray)):
        # if only single time delay as integer, make into a one element list
        tau = [tau]

    amis = np.zeros(len(tau))

    for i, t in enumerate(tau):
        y1 = y[:-t]
        y2 = y[t:]

        # Joint distribution of y1 and y2
        pij, _, _ = np.histogram2d(y1, y2, bins=(b, b))
        pij = pij[:numBins, :numBins]  # joint
        pij = pij / np.sum(pij)  # normalize
        pi = np.sum(pij, axis=1)  # marginal
        pj = np.sum(pij, axis=0)  # other marginal

        pii = np.tile(pi, (numBins, 1)).T
        pjj = np.tile(pj, (numBins, 1))

        r = pij > 0  # Defining the range in this way, we set log(0) = 0
        amis[i] = np.sum(pij[r] * np.log(pij[r] / pii[r] / pjj[r]))

    if len(tau) == 1:
        return amis[0]
    else:
        return {f'ami{i+1}': ami for i, ami in enumerate(amis)}


In [75]:
CO_HistogramAMI(ts3, 'ac', meth='quantiles')

0.04761744653583484

In [60]:
stats.kurtosis(ts3_zs, fisher=False)

3.0797345972733234

In [3]:
def stepBinary(X):
    # Transform real values to 0 if <=0 and 1 if >0:
    Y = np.zeros(len(X))
    Y[X > 0] = 1

    return Y

In [77]:
expFunc = lambda x, a, b : a * np.exp(b * x)

In [76]:
def firstUnder_fn(x, m, p):
    """
    Find the value of m for the first time p goes under the threshold, x. 
    p and m vectors of the same length
    """
    first_i = next((m_val for m_val, p_val in zip(m, p) if p_val < x), m[-1])
    return first_i

In [22]:
def BF_Binarize(y, binarizeHow='diff'):
    """
    """
    if binarizeHow == 'diff':
        # Binary signal: 1 for stepwise increases, 0 for stepwise decreases
        yBin = stepBinary(np.diff(y))
    
    elif binarizeHow == 'mean':
        # Binary signal: 1 for above mean, 0 for below mean
        yBin = stepBinary(y - np.mean(y))
    
    elif binarizeHow == 'median':
        # Binary signal: 1 for above median, 0 for below median
        yBin = stepBinary(y - np.median(y))
    
    elif binarizeHow == 'iqr':
        # Binary signal: 1 if inside interquartile range, 0 otherwise
        iqr = np.quantile(y,[.25,.75])
        iniqr = np.logical_and(y > iqr[0], y<iqr[1])
        yBin = np.zeros(len(y))
        yBin[iniqr] = 1
    else:
        raise ValueError(f"Unknown binary transformation setting '{binarizeHow}'")

    return yBin

In [25]:
mea = BF_Binarize(ts1, binarizeHow='mean')

In [27]:
mat = np.random.randn(3, 20)

In [31]:
np.size(mat, 1)

20

In [32]:
def BF_SignChange(y, doFind=0):
    """
    Where a data vector changes sign.

    """
    if doFind == 0:
        return (np.multiply(y[1:],y[0:len(y)-1]) < 0)
    indexs = np.where((np.multiply(y[1:],y[0:len(y)-1]) < 0))

    return indexs


In [36]:
out = BF_SignChange(ts1, doFind=1)

In [43]:
out

(array([ 14,  30,  46,  61,  77,  93, 108, 124, 140, 156, 171, 187, 203,
        218, 234, 250, 266, 281, 297, 313, 328, 344, 360, 375, 391, 407,
        423, 438, 454, 470, 485, 501, 517, 533, 548, 564, 580, 595, 611,
        627, 643, 658, 674, 690, 705, 721, 737, 752, 768, 784, 800, 815,
        831, 847, 862, 878, 894, 910, 925, 941, 957, 972, 988]),)

In [44]:
from scipy.stats import moment
import numpy as np

In [45]:
def DN_Moments(y, theMom):
    """
    A moment of the distribution of the input time series.
    
    """
    out = moment(y, theMom) / np.std(y) # normalized

    return out

In [51]:
DN_Moments(ts1, 5)

-0.005128976116081682

In [65]:
import numpy as np
from scipy import stats

def DN_Mean(y, mean_type='arithmetic'):
    """
    A given measure of location of a data vector.

    Parameters:
    y (array-like): The input data vector
    mean_type (str): The type of mean to calculate
        'norm' or 'arithmetic': arithmetic mean
        'median': median
        'geom': geometric mean
        'harm': harmonic mean
        'rms': root-mean-square
        'iqm': interquartile mean
        'midhinge': midhinge

    Returns:
    float: The calculated mean value

    Raises:
    ValueError: If an unknown mean type is specified
    """
    y = np.array(y)
    N = len(y)

    if mean_type in ['norm', 'arithmetic']:
        return np.mean(y)
    elif mean_type == 'median':
        return np.median(y)
    elif mean_type == 'geom':
        return stats.gmean(y)
    elif mean_type == 'harm':
        return stats.hmean(y)
    elif mean_type == 'rms':
        return np.sqrt(np.mean(y**2))
    elif mean_type == 'iqm':
        p = np.percentile(y, [25, 75])
        return np.mean(y[(y >= p[0]) & (y <= p[1])])
    elif mean_type == 'midhinge':
        p = np.percentile(y, [25, 75])
        return np.mean(p)
    else:
        raise ValueError(f"Unknown mean type '{mean_type}'")

In [66]:
def DN_ProportionValues(x, propWhat='positive'):

    N = len(x)

    if propWhat == 'zeros':
        # returns the proportion of zeros in the input vector
        out = sum(x == 0) / N
    elif propWhat == 'positive':
        out = sum(x > 0) / N
    elif propWhat == 'geq0':
        out = sum(x >= 0) / N
    else:
        raise ValueError(f"Unknown condition to measure: {propWhat}")

    return out


In [74]:
DN_ProportionValues(ts3, 'geq0')

0.519

In [75]:
def DN_Quantile(y, p=0.5):
    """
    Calculates the quantile value at a specified proportion, p.

    Parameters:
    y (array-like): The input data vector
    p (float): The quantile proportion (default is 0.5, which is the median)

    Returns:
    float: The calculated quantile value

    Raises:
    ValueError: If p is not a number between 0 and 1
    """
    if p == 0.5:
        print("Using quantile p = 0.5 (median) by default")
    
    if not isinstance(p, (int, float)) or p < 0 or p > 1:
        raise ValueError("p must specify a proportion, in (0,1)")
    
    return np.quantile(y, p)


In [82]:
DN_Quantile(ts1, p=0.3)

-0.58822

In [97]:
from scipy.stats import uniform, norm, geom

In [98]:
geom.fit(ts2)

AttributeError: 'geom_gen' object has no attribute 'fit'

In [107]:
def EN_CID(y):
    """
    Simple complexity measure of a time series.

    Estimates of 'complexity' of a time series as the stretched-out length of the
    lines resulting from a line-graph of the time series.

    Parameters:
    y (array-like): the input time series

    Returns:
    out (dict): 
    """
    CE1 = f_CE1(y)
    CE2 = f_CE2(y)

    minCE1 = f_CE1(np.sort(y))
    minCE2 = f_CE2(np.sort(y))

    CE1_norm = CE1 / minCE1
    CE2_norm = CE2 / minCE2

    out = {'CE1':CE1,'CE2':CE2,'minCE1':minCE1,'minCE2':minCE2,
            'CE1_norm':CE1_norm,'CE2_norm':CE2_norm}

    return out

def f_CE1(y):
    return np.sqrt(np.mean(np.power(np.diff(y),2)))

def f_CE2(y):
    return np.mean(np.sqrt(1 + np.power(np.diff(y),2)))


In [110]:
EN_CID(ts3)

{'CE1': 1.4397258123179755,
 'CE2': 1.623039534424403,
 'minCE1': 0.028789278256714637,
 'minCE2': 1.0003926815512982,
 'CE1_norm': 50.00909711872275,
 'CE2_norm': 1.6224024469147185}

In [111]:
def DN_Spread(y, spreadMeasure='std'):
    """
    Measure of spread of the input time series.
    Returns the spread of the raw data vector, as the standard deviation,
    inter-quartile range, mean absolute deviation, or median absolute deviation.
    """
    if spreadMeasure == 'std':
        out = np.std(y)
    elif spreadMeasure == 'iqr':
        out = stats.iqr(y)
    elif spreadMeasure == 'mad':
        out = mad(y)
    elif spreadMeasure == 'mead':
        out = mead(y)
    else:
        raise ValueError('spreadMeasure must be one of std, iqr, mad or mead')

    return out

def mad(data, axis=None):
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)

def mead(data, axis=None):
    return np.median(np.absolute(data - np.median(data, axis)), axis)


In [118]:
DN_Spread(ts2, spreadMeasure='mead')

0.66015

In [122]:
def DN_Unique(x):
    """
    The proportion of the time series that are unique values.

    Parameters:
    x (array-like): the input data vector

    Returns:
    out (float): the proportion of time series that are unique values
    """

    return len(np.unique(x)) / len(x)


In [123]:
DN_Unique(ts2)

0.977

In [127]:
len(np.unique(ts1))

897

In [148]:
def CO_NonlinearAutocorr(y,taus,doAbs ='empty'):

    if doAbs == 'empty':

        if len(taus) % 2 == 1:

            doAbs = 0

        else:

            doAbs = 1

    N = len(y)
    tmax = np.max(taus)

    nlac = y[tmax:N]

    for i in taus:

        nlac = np.multiply(nlac,y[ tmax - i:N - i ])

    if doAbs:

        return np.mean(np.absolute(nlac))

    else:

        return np.mean(nlac)

In [151]:
CO_NonlinearAutocorr(ts1, [1, 2, 3])

0.32868826608330426

In [153]:
from scipy.stats import trim_mean

In [190]:
def DN_TrimmedMean(y, n=0):
    """
    Mean of the trimmed time series using trimmean.

    Parameters:
    ----------
    y (array-like): the input time series
    n (float): the fraction of highest and lowest values in y to exclude from the mean calculation

    Returns:
    --------
    out (float): the mean of the trimmed time series.
    """
    n *= 0.01
    N = len(y)
    trim = int(np.round(N * n / 2))
    y = np.sort(y)

    out = np.mean(y[trim:N-trim])

    return out

In [193]:
DN_TrimmedMean(ts1, 10)

0.0023594444444444205

In [202]:
def DN_Burstiness(y):
    """
    Calculate the burstiness statistic of a time series.

    This function returns the 'burstiness' statistic as defined in
    Goh and Barabasi's paper, "Burstiness and memory in complex systems,"
    Europhys. Lett. 81, 48002 (2008).

    Parameters
    ----------
    y : array-like
        The input time series.
    
    Returns
    -------
    dict
        The original burstiness statistic, B, and the improved
        burstiness statistic, B_Kim.
    """
    
    mean = np.mean(y)
    std = np.std(y)

    r = np.divide(std,mean) # coefficient of variation
    B = np.divide((r - 1), (r + 1)) # Original Goh and Barabasi burstiness statistic, B

    # improved burstiness statistic, accounting for scaling for finite time series
    # Kim and Jo, 2016, http://arxiv.org/pdf/1604.01125v1.pdf
    N = len(y)
    p1 = np.sqrt(N+1)*r - np.sqrt(N-1)
    p2 = (np.sqrt(N+1)-2)*r + np.sqrt(N-1)

    B_Kim = np.divide(p1, p2)

    out = {'B': B, 'B_Kim': B_Kim}

    return out


In [203]:
DN_Burstiness(ts3)

{'B': 0.92662389689055, 'B_Kim': 0.9867869006600554}

In [445]:
import numpy as np
from Operations.CO_HistogramAMI import CO_HistogramAMI
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.IN_AutoMutualInfo import IN_AutoMutualInfo
from Operations.CO_AutoCorr import CO_AutoCorr
from PeripheryFunctions.BF_SignChange import BF_SignChange
from PeripheryFunctions.BF_iszscored import BF_iszscored
from scipy.optimize import curve_fit
import warnings

def CO_AddNoise(y, tau = 1, amiMethod = 'even', extraParam = None, randomSeed = None):
    """
    CO_AddNoise: Changes in the automutual information with the addition of noise

    Parameters:
    y (array-like): The input time series (should be z-scored)
    tau (int or str): The time delay for computing AMI (default: 1)
    amiMethod (str): The method for computing AMI:
                      'std1','std2','quantiles','even' for histogram-based estimation,
                      'gaussian','kernel','kraskov1','kraskov2' for estimation using JIDT
    extraParam: e.g., the number of bins input to CO_HistogramAMI, or parameter for IN_AutoMutualInfo
    randomSeed (int): Settings for resetting the random seed for reproducible results

    Returns:
    dict: Statistics on the resulting set of automutual information estimates
    """

    if not BF_iszscored(y):
        warnings.warn("Input time series should be z-scored")
    
    # Set tau to minimum of autocorrelation function if 'ac' or 'tau'
    if tau in ['ac', 'tau']:
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Generate noise
    if randomSeed is not None:
        np.random.seed(randomSeed)
    noise = np.random.randn(len(y)) # generate uncorrelated additive noise

    # Set up noise range
    noiseRange = np.linspace(0, 3, 50) # compare properties across this noise range
    numRepeats = len(noiseRange)

    # Compute the automutual information across a range of noise levels
    amis = np.zeros(numRepeats)
    if amiMethod in ['std1', 'std2', 'quantiles', 'even']:
        # histogram-based methods using my naive implementation in CO_Histogram
        for i in range(numRepeats):
            # use default num of bins for CO_HistogramAMI if not specified
            amis[i] = CO_HistogramAMI(y + noiseRange[i]*noise, tau, amiMethod, extraParam or 10)
            if np.isnan(amis[i]):
                raise ValueError('Error computing AMI: Time series too short (?)')
    if amiMethod in ['gaussian','kernel','kraskov1','kraskov2']:
        for i in range(numRepeats):
            amis[i] = IN_AutoMutualInfo(y + noiseRange[i]*noise, tau, amiMethod, extraParam)
            if np.isnan(amis[i]):
                raise ValueError('Error computing AMI: Time series too short (?)')
    
    # Output statistics
    out = {}
    # Proportion decreases
    out['pdec'] = np.sum(np.diff(amis) < 0) / (numRepeats - 1)

    # Mean change in AMI
    out['meanch'] = np.mean(np.diff(amis))

    # Autocorrelation of AMIs
    out['ac1'] = CO_AutoCorr(amis, 1, 'Fourier')[0]
    out['ac2'] = CO_AutoCorr(amis, 2, 'Fourier')[0]

    # Noise level required to reduce ami to proportion x of its initial value
    firstUnderVals = [0.75, 0.50, 0.25]
    for val in firstUnderVals:
        out[f'firstUnder{val*100}'] = firstUnder_fn(val * amis[0], noiseRange, amis)

    # AMI at actual noise levels: 0.5, 1, 1.5 and 2
    noiseLevels = [0.5, 1, 1.5, 2]
    for nlvl in noiseLevels:
        out[f'ami_at_{int(nlvl*10)}'] = amis[np.argmax(noiseRange >= nlvl)]

    # Count number of times the AMI function crosses its mean
    out['pcrossmean'] = np.sum(np.diff(np.sign(amis - np.mean(amis))) != 0) / (numRepeats - 1)

    # Fit exponential decay
    expFunc = lambda x, a, b : a * np.exp(b * x)
    popt, pcov = curve_fit(expFunc, noiseRange, amis, p0=[amis[0], -1])
    out['fitexpa'], out['fitexpb'] = popt
    residuals = amis - expFunc(noiseRange, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((amis - np.mean(amis))**2)
    out['fitexpr2'] = 1 - (ss_res / ss_tot)
    out['fitexpadjr2'] = 1 - (1-out['fitexpr2'])*(len(amis)-1)/(len(amis)-2-1)
    out['fitexprmse'] = np.sqrt(np.mean(residuals**2))

    # Fit linear function
    p = np.polyfit(noiseRange, amis, 1)
    out['fitlina'], out['fitlinb'] = p
    lin_fit = np.polyval(p, noiseRange)
    out['linfit_mse'] = np.mean((lin_fit - amis)**2)

    return out

# helper functions
def firstUnder_fn(x, m, p):
    """
    Find the value of m for the first time p goes under the threshold, x. 
    p and m vectors of the same length
    """
    first_i = next((m_val for m_val, p_val in zip(m, p) if p_val < x), m[-1])
    return first_i


In [456]:
import numpy as np
from scipy import stats
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from Operations.CO_AutoCorr import CO_AutoCorr
from Operations.CO_FirstCrossing import CO_FirstCrossing
from PeripheryFunctions.BF_SignChange import BF_SignChange
import warnings

def CO_AutoCorrShape(y, stopWhen = 'posDrown'):
    """
    CO_AutoCorrShape: How the autocorrelation function changes with the time lag.

    Outputs include the number of peaks, and autocorrelation in the
    autocorrelation function (ACF) itself.

    Parameters:
    -----------
    y : array_like
        The input time series
    stopWhen : str or int, optional
        The criterion for the maximum lag to measure the ACF up to.
        Default is 'posDrown'.

    Returns:
    --------
    dict
        A dictionary containing various metrics about the autocorrelation function.
    """
    N = len(y)

    # Only look up to when two consecutive values are under the significance threshold
    th = 2 / np.sqrt(N)  # significance threshold

    # Calculate the autocorrelation function, up to a maximum lag, length of time series (hopefully it's cropped by then)
    acf = []

    # At what lag does the acf drop to zero, Ndrown (by my definition)?
    if isinstance(stopWhen, int):
        taus = list(range(0, stopWhen+1))
        acf = CO_AutoCorr(y, taus, 'Fourier')
        Ndrown = stopWhen
    elif stopWhen in ['posDrown', 'drown', 'doubleDrown']:
        # Compute ACF up to a given threshold:
        Ndrown = 0 # the point at which ACF ~ 0
        if stopWhen == 'posDrown':
            # stop when ACF drops below threshold, th
            for i in range(1, N+1):
                acf_val = CO_AutoCorr(y, i-1, 'Fourier')[0]
                if np.isnan(acf_val):
                    warnings.warn("Weird time series (constant?)")
                    out = np.nan
                if acf_val < th:
                    # Ensure ACF is all positive
                    if acf_val > 0:
                        Ndrown = i
                        acf.append(acf_val)
                    else:
                        # stop at the previous point if not positive
                        Ndrown = i-1
                    # ACF has dropped below threshold, break the for loop...
                    break
                # hasn't dropped below thresh, append to list 
                acf.append(acf_val)
            # This should yield the initial, positive portion of the ACF.
            assert all(np.array(acf) > 0)
        elif stopWhen == 'drown':
            # Stop when ACF is very close to 0 (within threshold, th = 2/sqrt(N))
            for i in range(1, N+1):
                acf_val = CO_AutoCorr(y, i-1, 'Fourier')[0] # acf vector indicies are not lags
                # if positive and less than thresh
                if i > 0 and abs(acf_val) < th:
                    Ndrown = i
                    acf.append(acf_val)
                    break
                acf.append(acf_val)
        elif stopWhen == 'doubleDrown':
            # Stop at 2*tau, where tau is the lag where ACF ~ 0 (within 1/sqrt(N) threshold)
            for i in range(1, N+1):
                acf_val = CO_AutoCorr(y, i-1, 'Fourier')[0]
                if Ndrown > 0 and i == Ndrown * 2:
                    acf.append(acf_val)
                    break
                elif i > 1 and abs(acf_val) < th:
                    Ndrown = i
                acf.append(acf_val)
    else:
        raise ValueError(f"Unknown ACF decay criterion: '{stopWhen}'")

    acf = np.array(acf)
    Nac = len(acf)

    # Check for good behavior
    if np.any(np.isnan(acf)):
        # This is an anomalous time series (e.g., all constant, or conatining NaNs)
        out = np.NaN
    
    out = {}
    out['Nac'] = Ndrown

    # Basic stats on the ACF
    out['sumacf'] = np.sum(acf)
    out['meanacf'] = np.mean(acf)
    if stopWhen != 'posDrown':
        out['meanabsacf'] = np.mean(np.abs(acf))
        out['sumabsacf'] = np.sum(np.abs(acf))

    # Autocorrelation of the ACF
    minPointsForACFofACF = 5 # can't take lots of complex stats with fewer than this
    if Nac > minPointsForACFofACF:
        out['ac1'] = CO_AutoCorr(acf, 1, 'Fourier')[0]
        if all(acf > 0):
            out['actau'] = np.nan
        else:
            out['actau'] = CO_AutoCorr(acf, CO_FirstCrossing(acf, 'ac', 0, 'discrete'), 'Fourier')[0]
    else:
        out['ac1'] = np.nan
        out['actau'] = np.nan
    
    # Local extrema
    dacf = np.diff(acf)
    ddacf = np.diff(dacf)
    extrr = BF_SignChange(dacf, 1)
    sdsp = ddacf[extrr]

    # Proportion of local minima
    out['nminima'] = np.sum(sdsp > 0)
    out['meanminima'] = np.mean(sdsp[sdsp > 0])

    # Proportion of local maxima
    out['nmaxima'] = np.sum(sdsp < 0)
    out['meanmaxima'] = abs(np.mean(sdsp[sdsp < 0])) # must be negative: make it positive

    # Proportion of extrema
    out['nextrema'] = len(sdsp)
    out['pextrema'] = len(sdsp) / Nac

    # Fit exponential decay (only for 'posDrown', and if there are enough points)
    # Should probably only do this up to the first zero crossing...
    fitSuccess = False
    minPointsToFitExp = 4 # (need at least four points to fit exponential)
    if stopWhen == 'posDrown' and Nac >= minPointsToFitExp:
        # Fit exponential decay to (absolute) ACF:
        # (kind of only makes sense for the first positive period)
        expFunc = lambda x, b : np.exp(-b * x)
        try:
            popt, _ = curve_fit(expFunc, np.arange(Nac), acf, p0=0.5)
            fitSuccess = True
        except:
            fitSuccess = False
    if fitSuccess:
        bFit = popt[0] # fitted b
        out['decayTimescale'] = 1 / bFit
        expFit = expFunc(np.arange(Nac), bFit)
        residuals = acf - expFit
        out['fexpacf_r2'] = 1 - (np.sum(residuals**2) / np.sum((acf - np.mean(acf))**2))
        # had to fit a second exponential function with negative b to get same output as MATLAB for std residuals
        expFit2 = expFunc(np.arange(Nac), -bFit)
        residuals2 = acf - expFit2
        out['fexpacf_stdres'] = np.std(residuals2, ddof=1) # IMPORTANT *** DDOF=1 TO MATCH MATLAB STD ***
    else:
        # Fit inappropriate (or failed): return NaNs for the relevant stats
        out['decayTimescale'] = np.nan
        out['fexpacf_r2'] = np.nan
        out['fexpacf_stdres'] = np.nan
    
    return out


In [471]:
CO_AutoCorrShape(ts1, stopWhen=14)

{'Nac': 14,
 'sumacf': 1.745616526173292,
 'meanacf': 0.11637443507821947,
 'meanabsacf': 0.6154901825161944,
 'sumabsacf': 9.232352737742916,
 'ac1': 0.8415798606539007,
 'actau': -0.1164707090370352,
 'nminima': 0,
 'meanminima': nan,
 'nmaxima': 0,
 'meanmaxima': nan,
 'nextrema': 0,
 'pextrema': 0.0,
 'decayTimescale': nan,
 'fexpacf_r2': nan,
 'fexpacf_stdres': nan}

In [473]:
import numpy as np

def BF_MutualInformation(v1, v2, r1 = 'range', r2 = 'range', numBins = 10):
    """
    Compute mutual information between two data vectors using bin counting.

    Parameters:
    -----------
        v1 (array-like): The first input vector
        v2 (array-like): The second input vector
        r1 (str or list): The bin-partitioning method for v1 ('range', 'quantile', or [min, max])
        r2 (str or list): The bin-partitioning method for v2 ('range', 'quantile', or [min, max])
        numBins (int): The number of bins to partition each vector into (default : 10)

    Returns:
    --------
        float: The mutual information computed between v1 and v2
    """
    v1 = np.asarray(v1).flatten()
    v2 = np.asarray(v2).flatten()

    if len(v1) != len(v2):
        raise ValueError("Input vectors must be the same length")

    N = len(v1)

    # Create histograms
    edges_i = SUB_GiveMeEdges(r1, v1, numBins)
    edges_j = SUB_GiveMeEdges(r2, v2, numBins)

    ni, _ = np.histogram(v1, edges_i)
    nj, _ = np.histogram(v2, edges_j)

    # Create a joint histogram
    hist_xy, _, _ = np.histogram2d(v1, v2, [edges_i, edges_j])

    # Normalize counts to probabilities
    p_i = ni[:numBins] / N
    p_j = nj[:numBins] / N
    p_ij = hist_xy / N
    p_ixp_j = np.outer(p_i, p_j)

    # Calculate mutual information
    mask = (p_ixp_j > 0) & (p_ij > 0)
    if np.any(mask):
        mi = np.sum(p_ij[mask] * np.log(p_ij[mask] / p_ixp_j[mask]))
    else:
        print("The histograms aren't catching any points. Perhaps due to an inappropriate custom range for binning the data.")
        mi = np.nan

    return mi

def SUB_GiveMeEdges(r, v, nbins):
    EE = 1E-6 # this small addition gets lost in the last bin
    if r == 'range':
            return np.linspace(np.min(v), np.max(v) + EE, nbins + 1)
    elif r == 'quantile': # bin edges based on quantiles
        edges = np.quantile(v, np.linspace(0, 1, nbins + 1))
        edges[-1] += EE
        return edges
    elif isinstance(r, (list, np.ndarray)) and len(r) == 2: # a two-component vector
        return np.linspace(r[0], r[1] + EE, nbins + 1)
    else:
        raise ValueError(f"Unknown partitioning method '{r}'")


In [480]:
BF_MutualInformation(ts1, ts3, 'quantile', 'quantile', numBins=20)

0.18659001559071042

In [485]:
import numpy as np
from Operations.CO_AutoCorr import CO_AutoCorr
from Operations.IN_AutoMutualInfo import IN_AutoMutualInfo
from PeripheryFunctions.BF_MutualInformation import BF_MutualInformation
import warnings

def CO_FirstMin2(y, minWhat = 'mi-gaussian', extraParam = None, minNotMax = True):
    """
    Time of first minimum in a given self-correlation function.

    Parameters
    ----------
    y : array-like
        The input time series.
    minWhat : str, optional
        The type of correlation to minimize. Options are 'ac' for autocorrelation,
        or 'mi' for automutual information. By default, 'mi' specifies the
        'gaussian' method from the Information Dynamics Toolkit. Other options
        include 'mi-kernel', 'mi-kraskov1', 'mi-kraskov2' (from Information Dynamics Toolkit),
        or 'mi-hist' (histogram-based method). Default is 'mi'.
    extraParam : any, optional
        An additional parameter required for the specified `minWhat` method (e.g., for Kraskov).
    minNotMax : bool, optional
        If True, return the maximum instead of the minimum. Default is False.

    Returns
    -------
    int
        The time of the first minimum (or maximum if `minNotMax` is True).
    """

    N = len(y)

    # Define the autocorrelation function
    if minWhat in ['ac', 'corr']:
        # Autocorrelation implemented as CO_AutoCorr
        corrfn = lambda x : CO_AutoCorr(y, tau=x, method='Fourier')
    elif minWhat == 'mi-hist':
        # if extraParam is none, use default num of bins in BF_MutualInformation (default : 10)
        corrfn = lambda x : BF_MutualInformation(y[:-x], y[x:], 'range', 'range', extraParam or 10)
    elif minWhat == 'mi-kraskov2':
        # (using Information Dynamics Toolkit)
        # extraParam is the number of nearest neighbors
        corrfn = lambda x : IN_AutoMutualInfo(y, x, 'kraskov2', extraParam)
    elif minWhat == 'mi-kraskov1':
        # (using Information Dynamics Toolkit)
        corrfn = lambda x : IN_AutoMutualInfo(y, x, 'kraskov1', extraParam)
    elif minWhat == 'mi-kernel':
        corrfn = lambda x : IN_AutoMutualInfo(y, x, 'kernel', extraParam)
    elif minWhat in ['mi', 'mi-gaussian']:
        corrfn = lambda x : IN_AutoMutualInfo(y, x, 'gaussian', extraParam)
    else:
        raise ValueError(f"Unknown correlation type specified: {minWhat}")
    
    # search for a minimum (incrementally through time lags until a minimum is found)
    autoCorr = np.zeros(N-1) # pre-allocate maximum length autocorrelation vector
    if minNotMax:
        # FIRST LOCAL MINUMUM 
        for i in range(1, N):
            autoCorr[i-1] = corrfn(i)
            # Hit a NaN before got to a minimum -- there is no minimum
            if np.isnan(autoCorr[i-1]):
                warnings.warn(f"No minimum in {minWhat} [[time series too short to find it?]]")
                out = np.nan
            
            # we're at a local minimum
            if (i == 2) and (autoCorr[1] > autoCorr[0]):
                # already increases at lag of 2 from lag of 1: a minimum (since ac(0) is maximal)
                return 1
            elif (i > 2) and autoCorr[i-3] > autoCorr[i-2] < autoCorr[i-1]:
                # minimum at previous i
                return i-1 # I found the first minimum!
    else:
        # FIRST LOCAL MAXIMUM
        for i in range(1, N):
            autoCorr[i-1] = corrfn(i)
            # Hit a NaN before got to a max -- there is no max
            if np.isnan(autoCorr[i-1]):
                warnings.warn(f"No minimum in {minWhat} [[time series too short to find it?]]")
                return np.nan

            # we're at a local maximum
            if i > 2 and autoCorr[i-3] < autoCorr[i-2] > autoCorr[i-1]:
                return i-1

    return N


In [490]:
CO_FirstMin2(ts1, 'mi-hist', minNotMax=False)

9

In [491]:
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.CO_FirstMin import CO_FirstMin
import numpy as np

def CO_trev(y, tau = 'ac'):
    """
    Normalized nonlinear autocorrelation, trev function of a time series.

    Calculates the trev function, a normalized nonlinear autocorrelation,
    mentioned in the documentation of the TSTOOL nonlinear time-series analysis
    package.

    Parameters:
    y (array-like): Time series
    tau (int, str, optional): Time lag. Can be 'ac' or 'mi' to set as the first 
                              zero-crossing of the autocorrelation function, or 
                              the first minimum of the automutual information 
                              function, respectively. Default is 'ac'.

    Returns:
    dict: A dictionary containing the following keys:
        - 'raw': The raw trev expression
        - 'abs': The magnitude of the raw expression
        - 'num': The numerator
        - 'absnum': The magnitude of the numerator
        - 'denom': The denominator

    Raises:
    ValueError: If no valid setting for time delay is found.
    """

    # Can set the time lag, tau, to be 'ac' or 'mi'
    if tau == 'ac':
        # tau is first zero crossing of the autocorrelation function
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    elif tau == 'mi':
        # tau is the first minimum of the automutual information function
        tau = CO_FirstMin(y, 'mi')
    if np.isnan(tau):
        raise ValueError("No valid setting for time delay. (Is the time series too short?)")

    # Compute trev quantities
    yn = y[:-tau]
    yn1 = y[tau:] # yn, tau steps ahead
    
    out = {}

    # The trev expression used in TSTOOL
    raw = np.mean((yn1 - yn)**3) / (np.mean((yn1 - yn)**2))**(3/2)
    out['raw'] = raw

    # The magnitude
    out['abs'] = np.abs(raw)

    # The numerator
    num = np.mean((yn1-yn)**3)
    out['num'] = num
    out['absnum'] = np.abs(num)

    # the denominator
    out['denom'] = (np.mean((yn1-yn)**2))**(3/2)

    return out
